In [286]:
import polars as pl
#for lazy dataset processing

In [288]:
# Install dependencies as needed:
# pip install kagglehub[polars-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
for dirname, _, filenames in os.walk('/kaggle/input/nfl-combine-results-dataset-2000-2022'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nfl-combine-results-dataset-2000-2022/2016_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2011_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2022_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2021_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2015_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2004_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2019_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2012_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2005_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2003_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2010_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2007_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2008_combine.csv
/kaggle/input/nfl-combine-results-dataset-2000-2022/2006_combine.csv
/kaggle/input/nfl-combine-results-

In [287]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-big-data-bowl-2026-prediction/test_input.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/test.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/nfl_inference_server.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/nfl_gateway.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/__init__.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/templates.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/base_gateway.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/relay.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/__init__.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/generated/kaggle_evaluation_pb2.py
/kaggle/input/nfl-big-data-bowl-2026-prediction/kaggle_evaluation/core/generated/kaggle_evaluati

In [289]:
path = kagglehub.dataset_download("benhorne00/2025-nfl-combine-dataset")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/2025-nfl-combine-dataset


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [291]:
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [292]:
class EncoderDecoderRNN(nn.Module):
    def __init__(self, encoder_input_dim, static_dim, hidden_dim, 
                 max_output_len=50, num_layers=2):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # ENCODER
        self.encoder = nn.LSTM(
            input_size=encoder_input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2 if num_layers > 1 else 0
        )
        
        # ATTENTION
        self.attention = Attention(hidden_dim)
        
        # POSITIONAL ENCODING
        self.positional_encoding = LearnedPositionalEmbedding(
            max_len=max_output_len,
            embedding_dim=32
        )
        
        # DECODER
        decoder_input_size = 2 + static_dim + hidden_dim + 32
        
        self.decoder = nn.LSTM(
            input_size=decoder_input_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2 if num_layers > 1 else 0
        )
        
        # Output layer
        self.fc = nn.Linear(hidden_dim, 2)
    
    def forward(self, encoder_input, static_features, num_output_frames,
                decoder_target=None, teacher_forcing_ratio=0.5):  # ← 4 spaces, not 8!
        batch_size = encoder_input.size(0)
        device = encoder_input.device
        
        # ENCODE
        encoder_outputs, (hidden, cell) = self.encoder(encoder_input)
        
        # Start with last position
        decoder_input = encoder_input[:, -1, :2]
        
        # Prepare max output frames
        if isinstance(num_output_frames, int):
            max_output_frames = num_output_frames
        elif torch.is_tensor(num_output_frames):
            max_output_frames = num_output_frames.item() if num_output_frames.dim() == 0 else int(max(num_output_frames))
        else:
            max_output_frames = int(max(num_output_frames))
        
        outputs = []
        attention_weights_list = []
        
        for t in range(max_output_frames):
            # Get attention context using top decoder hidden state
            decoder_hidden_top = hidden[-1]  # (batch, hidden_dim)
            context, attn_weights = self.attention(encoder_outputs, decoder_hidden_top)
            
            # Get positional encoding for current timestep
            timestep_tensor = torch.full((batch_size,), t, dtype=torch.long, device=device)
            pos_encoding = self.positional_encoding(timestep_tensor)  # (batch, 32)
            
            # Concatenate all decoder inputs
            decoder_step_input = torch.cat([
                decoder_input,      # (batch, 2)
                static_features,    # (batch, static_dim)
                context,           # (batch, hidden_dim)
                pos_encoding       # (batch, 32)
            ], dim=1).unsqueeze(1)  # (batch, 1, decoder_input_size)
            
            # Decoder step
            decoder_output, (hidden, cell) = self.decoder(
                decoder_step_input, (hidden, cell)
            )
            
            # Predict next position
            prediction = self.fc(decoder_output.squeeze(1))  # (batch, 2)
            outputs.append(prediction)
            attention_weights_list.append(attn_weights)
            
            # Teacher forcing
            if decoder_target is not None and t < decoder_target.size(1):
                if torch.rand(1).item() < teacher_forcing_ratio:
                    decoder_input = decoder_target[:, t, :]
                else:
                    decoder_input = prediction
            else:
                decoder_input = prediction
        
        # Stack outputs
        predictions = torch.stack(outputs, dim=1)  # (batch, max_output_frames, 2)
        attention_weights = torch.stack(attention_weights_list, dim=1)
        
        return predictions, attention_weights

In [293]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        
        # Learnable weights
        self.W_encoder = nn.Linear(hidden_dim, hidden_dim)
        self.W_decoder = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        
    def forward(self, encoder_outputs, decoder_hidden):
        """
        encoder_outputs: (batch, seq_len, hidden_dim)
        decoder_hidden: (batch, hidden_dim)
        
        Returns:
        context: (batch, hidden_dim)
        attention_weights: (batch, seq_len)
        """
        seq_len = encoder_outputs.size(1)
        
        # Expand decoder hidden to match encoder seq_len
        decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, seq_len, 1)
        # (batch, seq_len, hidden_dim)
        
        # Calculate attention scores
        energy = torch.tanh(
            self.W_encoder(encoder_outputs) + self.W_decoder(decoder_hidden)
        )  # (batch, seq_len, hidden_dim)
        
        attention_scores = self.V(energy).squeeze(-1)  # (batch, seq_len)
        
        # Softmax to get attention weights
        attention_weights = torch.softmax(attention_scores, dim=1)
        # (batch, seq_len)
        
        # Weighted sum of encoder outputs
        context = torch.bmm(
            attention_weights.unsqueeze(1),  # (batch, 1, seq_len)
            encoder_outputs  # (batch, seq_len, hidden_dim)
        ).squeeze(1)  # (batch, hidden_dim)
        
        return context, attention_weights

In [294]:
class LearnedPositionalEmbedding(nn.Module):
    def __init__(self, max_len, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(max_len, embedding_dim)
        
    def forward(self, timestep):
        """
        timestep: int or (batch,) tensor of timesteps
        """
        if isinstance(timestep, int):
            timestep = torch.tensor([timestep], device=self.embedding.weight.device)
        return self.embedding(timestep)

In [295]:
class EncoderDecoderDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = self.sequences[idx]
        return {
            'encoder_input': torch.FloatTensor(seq['encoder_input']),
            'decoder_target': torch.FloatTensor(seq['decoder_target']),
            'static_features': torch.FloatTensor(seq['static_features']),
            'num_input_frames': seq['num_input_frames'],
            'num_output_frames': seq['num_output_frames']
        }

def collate_fn_encoder_decoder(batch):
    """Handle variable length sequences"""
    # Find max lengths
    max_input_len = max(item['num_input_frames'] for item in batch)
    max_output_len = max(item['num_output_frames'] for item in batch)
    
    batch_size = len(batch)
    encoder_input_dim = batch[0]['encoder_input'].shape[1]
    static_dim = batch[0]['static_features'].shape[0]
    
    # Initialize padded tensors
    padded_encoder_input = torch.zeros(batch_size, max_input_len, encoder_input_dim)
    padded_decoder_target = torch.zeros(batch_size, max_output_len, 2)
    static_features = torch.zeros(batch_size, static_dim)
    
    input_lengths = []
    output_lengths = []
    
    # Fill in data
    for i, item in enumerate(batch):
        input_len = item['num_input_frames']
        output_len = item['num_output_frames']
        
        padded_encoder_input[i, :input_len, :] = item['encoder_input']
        padded_decoder_target[i, :output_len, :] = item['decoder_target']
        static_features[i] = item['static_features']
        
        input_lengths.append(input_len)
        output_lengths.append(output_len)
    
    return {
        'encoder_input': padded_encoder_input,
        'decoder_target': padded_decoder_target,
        'static_features': static_features,
        'input_lengths': torch.LongTensor(input_lengths),
        'output_lengths': torch.LongTensor(output_lengths)
    }

# Training loop adjustment
def train_epoch(model, dataloader, optimizer, device, teacher_forcing_ratio=0.5):
    model.train()
    total_loss = 0
    
    for batch in tqdm(dataloader, desc="Training"):
        encoder_input = batch['encoder_input'].to(device)
        decoder_target = batch['decoder_target'].to(device)
        static_features = batch['static_features'].to(device)
        output_lengths = batch['output_lengths'].to(device)
        
        optimizer.zero_grad()
        
        # Forward with teacher forcing - UNPACK BOTH RETURNS
        predictions, attention_weights = model(
            encoder_input, 
            static_features,
            max(output_lengths),
            decoder_target=decoder_target,
            teacher_forcing_ratio=teacher_forcing_ratio
        )
        
        # Masked loss (just use predictions, ignore attention_weights for now)
        loss = masked_mse_loss(predictions, decoder_target, output_lengths)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [296]:
def masked_mse_loss(predictions, targets, lengths):
    """
    Compute MSE loss only on valid (non-padded) positions
    
    predictions: (batch, max_seq_len, 2)
    targets: (batch, max_seq_len, 2)
    lengths: (batch,) - actual lengths for each sequence
    """
    batch_size = predictions.size(0)
    max_len = predictions.size(1)
    
    # Create mask: 1 for valid positions, 0 for padding
    mask = torch.arange(max_len, device=predictions.device).expand(batch_size, max_len)
    mask = (mask < lengths.unsqueeze(1)).float().unsqueeze(-1)  # (batch, max_len, 1)
    
    # Compute squared error
    squared_error = (predictions - targets) ** 2
    
    # Apply mask and compute mean
    masked_error = squared_error * mask
    loss = masked_error.sum() / mask.sum()
    
    return loss

In [297]:
def validate_epoch(model, dataloader, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            encoder_input = batch['encoder_input'].to(device)
            decoder_target = batch['decoder_target'].to(device)
            static_features = batch['static_features'].to(device)
            output_lengths = batch['output_lengths'].to(device)
            
            # UNPACK BOTH RETURNS
            predictions, attention_weights = model(
                encoder_input,
                static_features,
                max(output_lengths),
                decoder_target=None,  # No teacher forcing during validation
                teacher_forcing_ratio=0.0
            )
            
            loss = masked_mse_loss(predictions, decoder_target, output_lengths)
            total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [298]:
train_path = "/kaggle/input/nfl-big-data-bowl-2026-prediction/train/"
test_path  = "/kaggle/input/nfl-big-data-bowl-2026-prediction/"

import glob

# Barcha train input fayllarni birlashtirish
input_files = sorted(glob.glob(train_path + "input_2023_w*.csv"))
output_files = sorted(glob.glob(train_path + "output_2023_w*.csv"))

train_inputs = pd.concat([pd.read_csv(f) for f in input_files], ignore_index=True)
train_outputs = pd.concat([pd.read_csv(f) for f in output_files], ignore_index=True)

print(train_inputs.shape, train_outputs.shape)

(4880579, 23) (562936, 6)


In [299]:
def train_model(model, train_loader, val_loader, optimizer, scheduler, 
                num_epochs, device, save_path='best_model.pth'):
    """
    Complete training loop with validation and model saving
    """
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        print(f"\n{'='*50}")
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"{'='*50}")
        
        # Train
        train_loss = train_epoch(
            model, train_loader, optimizer, device,
            teacher_forcing_ratio=0.5  # Can decay this over epochs
        )
        train_losses.append(train_loss)
        
        # Validate
        val_loss = validate_epoch(model, val_loader, device)
        val_losses.append(val_loss)
        
        # Learning rate scheduling
        if scheduler is not None:
            scheduler.step(val_loss)
        
        print(f"Train Loss: {train_loss:.6f}")
        print(f"Val Loss:   {val_loss:.6f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
            }, save_path)
            print(f"✓ Saved best model (val_loss: {val_loss:.6f})")
    
    return train_losses, val_losses

In [300]:
test_input = pd.read_csv(test_path + "test_input.csv")
test_data  = pd.read_csv(test_path + "test.csv")

In [301]:
test_input.describe()

,game_id,play_id,nfl_id,frame_id,absolute_yardline_number,player_weight,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
count,4.975300e+04,49753.000000,49753.00000,49753.000000,49753.000000,49753.000000,49753.000000,49753.000000,49753.000000,49753.000000,49753.000000,49753.000000,49753.000000,49753.000000,49753.000000
mean,2.024382e+09,2092.477760,51161.28143,16.011718,57.507185,210.669367,57.934714,26.721885,3.008711,2.049136,175.150532,183.756428,11.524029,59.113787,27.115044
std,4.050343e+05,1198.481594,5506.75297,10.568196,24.055066,22.353352,24.241947,9.792361,2.151394,1.287678,101.101264,98.530383,4.799505,25.126137,14.603694
min,2.024121e+09,63.000000,38588.00000,1.000000,11.000000,165.000000,1.710000,0.980000,0.000000,0.000000,0.010000,0.000000,5.000000,2.540000,-0.830000
25%,2.024121e+09,1201.000000,46142.00000,8.000000,40.000000,195.000000,39.970000,19.120000,1.150000,1.040000,86.810000,94.430000,8.000000,43.770000,14.650000
50%,2.024122e+09,2090.000000,53496.00000,15.000000,60.000000,205.000000,59.890000,26.730000,2.750000,1.890000,169.880000,187.570000,11.000000,63.070000,27.190001
75%,2.025011e+09,3156.000000,55949.00000,22.000000,79.000000,228.000000,75.530000,34.290000,4.550000,2.920000,266.470000,273.290000,13.000000,75.900002,38.779999
max,2.025011e+09,4154.000000,57801.00000,60.000000,109.000000,347.000000,119.360000,51.390000,9.560000,9.200000,359.990000,359.990000,30.000000,117.099998,52.849998


In [302]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [303]:
def processPolarsOutputs(test_polars, combine_polars):
    test_polars = test_polars.with_columns(
        (pl.col("y") / 53.3).alias("y"),
        (pl.col("x") / 120).alias("x")
    )
    return test_polars

In [304]:
def processPolars(test_polars, combine_polars):
    test_polars = test_polars.with_columns(
        [pl.col("player_birth_date").str.to_datetime(format="%Y-%m-%d").alias("datetime"),
    ]
    ).drop("player_birth_date").with_columns(
        [pl.col("datetime").dt.year().alias("player_born_year")]
    ).drop("datetime").with_columns(
        encoded_play_direction=pl.col("play_direction").str.encode("hex")
    ).drop("play_direction").with_columns(
        play_direction=pl.col("encoded_play_direction").str.to_integer(base=16, strict=False)
    ).drop("encoded_play_direction").with_columns(
        test_polars["player_role"].to_dummies()
    ).drop("player_role").with_columns(
        test_polars["player_position"].to_dummies()
    ).drop("player_position").with_columns(
        test_polars["player_side"].to_dummies()
    ).drop("player_side").with_columns(
        pl.col("player_height").str.extract(r"(\d+)-", 1).cast(pl.Int32).alias("feet"),
        pl.col("player_height").str.extract(r"-(\d+)", 1).cast(pl.Int32).alias("inches"),
    ).with_columns(
        (pl.col("feet") * 12 + pl.col("inches") + 2).alias("height")
    ).drop("player_height").drop("inches").drop("feet")
    
    # Define expected columns from training
    EXPECTED_POSITION_COLS = [
        'player_position_CB', 'player_position_DE', 'player_position_DT',
        'player_position_FB', 'player_position_FS', 'player_position_ILB',
        'player_position_MLB', 'player_position_NT', 'player_position_OLB',
        'player_position_QB', 'player_position_RB', 'player_position_S',
        'player_position_SS', 'player_position_TE', 'player_position_WR'
    ]
    
    EXPECTED_ROLE_COLS = [
        'player_role_Defensive Coverage', 'player_role_Other Route Runner',
        'player_role_Passer', 'player_role_Targeted Receiver'
    ]
    
    EXPECTED_SIDE_COLS = ['player_side_Defense', 'player_side_Offense']
    
    # Remove any extra columns not in training
    for col in test_polars.columns:
        if col.startswith('player_position_') and col not in EXPECTED_POSITION_COLS:
            test_polars = test_polars.drop(col)
        if col.startswith('player_role_') and col not in EXPECTED_ROLE_COLS:
            test_polars = test_polars.drop(col)
        if col.startswith('player_side_') and col not in EXPECTED_SIDE_COLS:
            test_polars = test_polars.drop(col)
    
    # Add missing columns as zeros
    for col in EXPECTED_POSITION_COLS + EXPECTED_ROLE_COLS + EXPECTED_SIDE_COLS:
        if col not in test_polars.columns:
            test_polars = test_polars.with_columns(pl.lit(0).alias(col))
    
    # Now join with combine data
    combined_test_polars = test_polars.join(
        combine_polars, on="player_name", how="full"
    ).drop("player_name_right").with_columns(
        encoded_player_name=pl.col("player_name").str.encode("hex")
    ).with_columns(
        player_name=pl.col("encoded_player_name").str.to_integer(base=16, strict=False)
    ).drop("encoded_player_name").drop("Ht").drop("Pos").drop("player_weight").with_columns(
        (pl.col("y") / 53.3).alias("y"),
        (pl.col("x") / 120).alias("x"),
        (pl.col("ball_land_x") / 120).alias("ball_land_x"),
        (pl.col("ball_land_y") / 53.3).alias("ball_land_y"),
    ).drop("School")
    
    return combined_test_polars

In [305]:
# Barcha train input fayllarni birlashtirish
train_path = '/kaggle/input/nfl-combine-results-dataset-2000-2022/'
combine_files = sorted(glob.glob(train_path + "*_combine.csv"))

combine_data = pd.concat([pd.read_csv(f) for f in combine_files], ignore_index=True)
#loads the combine data
combine_polars = pl.from_pandas(combine_data)
combine_polars = combine_polars.rename({'Player': 'player_name'})

In [306]:
test_polars = processPolars(pl.from_pandas(test_input),combine_polars)
train_input_polars = processPolars(pl.from_pandas(train_inputs),combine_polars)

In [307]:
test_data_polars = pl.from_pandas(test_data)
train_output_polars = processPolarsOutputs(pl.from_pandas(train_outputs),combine_polars)

In [308]:
train_output_polars.describe()

statistic,game_id,play_id,nfl_id,frame_id,x,y
str,f64,f64,f64,f64,f64,f64
"""count""",562936.0,562936.0,562936.0,562936.0,562936.0,562936.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",2.0232e9,2219.785841,49647.292742,7.865402,0.502148,0.499407
"""std""",202018.858446,1246.443129,5089.634979,5.923855,0.210744,0.252274
"""min""",2.0231e9,54.0,30842.0,1.0,0.000167,0.006191
"""25%""",2.0231e9,1185.0,45395.0,4.0,0.358333,0.279737
"""50%""",2.0231e9,2207.0,52423.0,7.0,0.500667,0.496248
"""75%""",2.0231e9,3278.0,54496.0,11.0,0.644333,0.7197
"""max""",2.0240e9,5258.0,56673.0,94.0,1.006917,1.00788


In [309]:
train_input_polars.describe()

statistic,game_id,play_id,player_to_predict,nfl_id,frame_id,absolute_yardline_number,player_name,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y,player_born_year,play_direction,player_role_Defensive Coverage,player_role_Other Route Runner,player_role_Passer,player_role_Targeted Receiver,player_position_CB,player_position_DE,player_position_DT,player_position_FB,player_position_FS,player_position_ILB,player_position_MLB,player_position_NT,player_position_OLB,player_position_QB,player_position_RB,player_position_S,player_position_SS,player_position_TE,player_position_WR,player_side_Defense,player_side_Offense,height,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,29368.0,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,4.94172e6,3.356672e6,2.94157e6,2.768143e6,2.060714e6,2.717257e6,1.887875e6,1.956297e6
"""null_count""",6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,4.919178e6,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,6826.0,1.591874e6,2.006976e6,2.180403e6,2.887832e6,2.231289e6,3.060671e6,2.992249e6
"""mean""",2.0232e9,2194.49186,0.266164,49510.873115,16.136608,60.549741,5.1929e18,0.504177,0.503034,3.012435,2.115664,180.508896,181.631006,11.643935,0.504268,0.499675,1996.758192,2.4846e11,0.545077,0.289301,0.085022,0.0806,0.214003,0.003445,0.000635,0.004222,0.100315,0.062074,0.040468,0.000244,0.041975,0.08588,0.063726,0.002785,0.079425,0.084413,0.216356,0.545077,0.454923,75.079607,213.250433,4.542937,35.452883,16.929729,122.25434,6.970367,4.237323
"""std""",201188.201408,1246.494217,null,5205.537597,11.136018,23.062136,3.1525e17,0.195803,0.187099,2.224392,1.41397,100.705848,98.01594,5.332433,0.21083,0.289732,2.974658,2.4478e11,0.497964,0.453438,0.278914,0.27222,0.410129,0.058589,0.025195,0.06484,0.30042,0.241289,0.197055,0.015633,0.200532,0.280187,0.244265,0.052702,0.270401,0.278006,0.41176,0.497964,0.497964,2.255597,22.720532,0.139525,3.137675,4.748011,6.225874,0.215392,0.148692
"""min""",2.0231e9,54.0,0.0,30842.0,1.0,11.0,4.7844e18,0.003417,0.011632,0.0,0.0,0.0,0.0,5.0,-0.043833,-0.073358,1984.0,1.8186e9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68.0,144.0,4.22,17.5,2.0,74.0,6.28,3.73
"""25%""",2.0231e9,1148.0,null,45185.0,8.0,41.0,4.9208e18,0.35525,0.357223,1.09,1.01,90.91,91.8,8.0,0.355083,0.249343,1995.0,1.8186e9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,197.0,4.45,33.5,14.0,118.0,6.82,4.13
"""50%""",2.0231e9,2170.0,null,49410.0,15.0,61.0,5.3559e18,0.5035,0.503565,2.71,1.92,179.61,180.19,10.0,0.50425,0.496248,1997.0,4.9139e11,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,75.0,209.0,4.52,35.5,17.0,122.0,6.96,4.23
"""75%""",2.0231e9,3244.0,null,54496.0,22.0,80.0,5.4317e18,0.652,0.648593,4.6,3.04,270.84,271.66,14.0,0.653917,0.74803,1999.0,4.9139e11,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,77.0,229.0,4.62,37.5,19.0,126.0,7.09,4.34
"""max""",2.0240e9,5258.0,1.0,56673.0,123.0,109.0,6.2266e18,0.998833,0.99212,12.53,17.12,360.0,360.0,94.0,1.04875,1.07561,2002.0,4.9139e11,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,83.0,384.0,6.05,46.5,49.0,147.0,9.12,5.56


In [310]:
test_polars.describe()

statistic,game_id,play_id,player_to_predict,nfl_id,frame_id,absolute_yardline_number,player_name,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y,player_born_year,play_direction,player_role_Defensive Coverage,player_role_Other Route Runner,player_role_Passer,player_role_Targeted Receiver,player_position_CB,player_position_DE,player_position_DT,player_position_FB,player_position_FS,player_position_ILB,player_position_MLB,player_position_NT,player_position_OLB,player_position_QB,player_position_RB,player_position_S,player_position_SS,player_position_TE,player_position_WR,player_side_Defense,player_side_Offense,height,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,327.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,50013.0,32632.0,30530.0,25664.0,22290.0,24704.0,20876.0,22199.0
"""null_count""",7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,57305.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,7619.0,25000.0,27102.0,31968.0,35342.0,32928.0,36756.0,35433.0
"""mean""",2.0244e9,2083.600244,0.267231,51193.516786,16.014476,57.504089,6.1433e18,0.482949,0.501501,3.006484,2.048644,175.095483,184.27248,11.516846,0.492752,0.509933,1997.751885,2.6802e11,0.549857,0.288785,0.080679,0.080679,0.188651,0.005818,0.00162,0.002119,0.111591,0.062804,0.026793,0.00032,0.062704,0.080679,0.062964,0.005099,0.084458,0.096415,0.207966,0.549857,0.450143,75.046568,219.228947,4.598401,35.18081,16.802557,120.673858,7.053552,4.259298
"""std""",406498.311502,1201.755475,null,5510.49243,10.570484,24.072299,3.4544e17,0.202122,0.183523,2.148734,1.287437,101.165637,98.540441,4.791391,0.209505,0.27439,3.083971,2.4385e11,0.497513,0.453202,0.272344,0.272344,0.391235,0.076058,0.040212,0.045989,0.314866,0.242612,0.16148,0.017884,0.242432,0.272344,0.2429,0.071223,0.278076,0.295162,0.405856,0.497513,0.497513,2.416545,31.507253,0.21363,3.844182,4.949528,8.060606,0.289234,0.199295
"""min""",2.0241e9,63.0,0.0,38588.0,1.0,11.0,4.7143e18,0.01425,0.018386,0.0,0.0,0.01,0.0,5.0,0.021167,-0.015572,1990.0,1.8186e9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,144.0,4.22,17.5,2.0,74.0,6.28,3.73
"""25%""",2.0241e9,1130.0,null,46142.0,8.0,40.0,6.2265e18,0.333083,0.358912,1.15,1.04,86.71,94.59,8.0,0.36475,0.274859,1995.0,1.8186e9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,199.0,4.48,32.5,14.0,117.0,6.93,4.15
"""50%""",2.0241e9,2090.0,null,53496.0,15.0,60.0,6.2265e18,0.499,0.501876,2.76,1.89,169.65,188.89,11.0,0.525583,0.52758,1998.0,4.9139e11,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,75.0,210.0,4.54,35.5,15.0,120.0,7.0,4.23
"""75%""",2.0250e9,3156.0,null,55949.0,22.0,79.0,6.2265e18,0.62975,0.642964,4.55,2.92,266.49,273.53,13.0,0.6325,0.72758,2000.0,4.9139e11,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,77.0,234.0,4.64,37.5,19.0,126.0,7.12,4.35
"""max""",2.0250e9,4154.0,1.0,57801.0,60.0,109.0,6.2266e18,0.994667,0.964165,9.56,9.2,359.99,359.99,30.0,0.975833,0.991557,2003.0,4.9139e11,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,81.0,384.0,6.05,46.5,49.0,147.0,9.12,5.56


In [311]:
def create_sequences(input_df, output_df):
    """
    Create encoder-decoder sequences from input and output dataframes
    
    Args:
        input_df: Tracking data BEFORE ball is thrown (encoder input)
        output_df: Tracking data WHILE ball is in air (decoder target)
    
    Returns:
        List of sequence dictionaries
    """
    # Define feature columns
    temporal_features = ['x', 'y', 's', 'a', 'dir', 'o']
    
    static_feature_cols = [
        'height', 'Wt', '40yd', 'Vertical', 'Bench', 
        'Broad Jump', '3Cone', 'Shuttle', 'player_born_year',
        'absolute_yardline_number', 'ball_land_x', 'ball_land_y',
        'play_direction'
    ] + [col for col in input_df.columns if col.startswith(
        ('player_position_', 'player_role_', 'player_side_'))]
    
    # Convert to lazy if not already
    input_lazy = input_df.lazy() if not isinstance(input_df, pl.LazyFrame) else input_df
    output_lazy = output_df.lazy() if not isinstance(output_df, pl.LazyFrame) else output_df
    
    # Sort both dataframes
    input_sorted = input_lazy.sort(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    output_sorted = output_lazy.sort(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    
    # Group columns
    group_cols = ['game_id', 'play_id', 'nfl_id']

    #works since data is now in temporal format
    
    # Pre-compute aggregations using Polars' efficient group_by
    input_agg = input_sorted.group_by(group_cols, maintain_order=True).agg([
        *[pl.col(feat).alias(f'temporal_{feat}') for feat in temporal_features],
        *[pl.col(col).first().alias(f'{col}_static') for col in static_feature_cols],
        pl.len().alias('num_input_frames')
    ])
    
    output_agg = output_sorted.group_by(group_cols, maintain_order=True).agg([
        pl.col('x').alias('decoder_x'),
        pl.col('y').alias('decoder_y'),
        pl.len().alias('num_output_frames')
    ])
    
    # Join the aggregated data and collect
    joined = input_agg.join(output_agg, on=group_cols, how='inner').collect()
    
    # Convert to sequences
    sequences = []
    for row in joined.iter_rows(named=True):
        # Extract temporal data - each feature is a list
        temporal_data = np.column_stack([
            row[f'temporal_{feat}'] for feat in temporal_features
        ])
        
        # Extract static features
        static_data = np.array([row[f'{col}_static'] for col in static_feature_cols], dtype=np.float32)
        
        # Repeat static features for each input frame
        num_input_frames = row['num_input_frames']
        static_repeated = np.tile(static_data, (num_input_frames, 1))
        
        # Combine temporal + static
        encoder_input = np.concatenate([temporal_data, static_repeated], axis=1)
        
        # Decoder target
        decoder_target = np.column_stack([
            row['decoder_x'],
            row['decoder_y']
        ])
        
        sequences.append({
            'encoder_input': encoder_input,
            'decoder_target': decoder_target,
            'static_features': static_data,
            'num_input_frames': num_input_frames,
            'num_output_frames': row['num_output_frames'],
            'game_id': row['game_id'],
            'play_id': row['play_id'],
            'nfl_id': row['nfl_id']
        })
    
    return sequences

In [312]:
from sklearn.preprocessing import StandardScaler
#scale data

In [313]:
# Fill any None/null values
train_input_polars = train_input_polars.fill_null(0)
train_output_polars = train_output_polars.fill_null(0)
sequences = create_sequences(train_input_polars,train_output_polars)
print('one line')

one line


In [314]:
from sklearn.model_selection import train_test_split
alpha = 0.001 #learning rate
batch_size = 32 #?
hidden_size = 64
input_size = 41
num_layers = 2
num_classes = 2

In [315]:
train_sequences, val_sequences = train_test_split(sequences, test_size=0.2, random_state=42)

In [316]:
train_dataset = EncoderDecoderDataset(train_sequences)
val_dataset = EncoderDecoderDataset(val_sequences)

In [317]:
train_loader = DataLoader(train_dataset, batch_size, shuffle=True,
                         collate_fn=collate_fn_encoder_decoder, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size, shuffle=False,
                       collate_fn=collate_fn_encoder_decoder, num_workers=2)

In [323]:
sample = train_sequences[0]
encoder_input_dim = sample['encoder_input'].shape[1]
static_dim = len(sample['static_features'])

In [324]:
max_frames_in_data = 0
for batch in train_loader:
    max_frames_in_data = max(max_frames_in_data, max(batch['output_lengths']))

print(f"Max frames in dataset: {max_frames_in_data}")

# Create model with buffer
model = EncoderDecoderRNN(
    encoder_input_dim=encoder_input_dim,
    static_dim=static_dim,
    hidden_dim=128,
    max_output_len=max_frames_in_data + 10,  # Add buffer
    num_layers=2
).to(device)

Max frames in dataset: 94


In [325]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3
)

In [326]:
#'''
print("\n" + "="*60)
print("TRAINING")
print("="*60)

best_val_loss = float('inf')
train_losses = []
val_losses = []

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    print("-" * 60)
    
    # Train and validate
    train_loss = train_epoch(model, train_loader, optimizer, device, 
                            teacher_forcing_ratio=0.5)
    val_loss = validate_epoch(model, val_loader, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    scheduler.step(val_loss)
    
    print(f"Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'encoder_input_dim': encoder_input_dim,
            'static_dim': static_dim,
            'hidden_dim': HIDDEN_DIM,
            'num_layers': NUM_LAYERS
        }, 'best_nfl_model.pth')
        print(f"✓ Saved best model")

print("\n" + "="*60)
print(f"TRAINING COMPLETE! Best Val Loss: {best_val_loss:.6f}")
print("="*60)
#'''


TRAINING

Epoch 1/10
------------------------------------------------------------


Validation: 100%|██████████| 288/288 [00:38<00:00,  7.52it/s]


Train Loss: 0.111012 | Val Loss: 0.108446
✓ Saved best model

Epoch 2/10
------------------------------------------------------------


Validation: 100%|██████████| 288/288 [00:37<00:00,  7.67it/s]


Train Loss: 0.109164 | Val Loss: 0.109836

Epoch 3/10
------------------------------------------------------------


...

Train Loss: 0.108919 | Val Loss: 0.107568
✓ Saved best model

Epoch 8/10
------------------------------------------------------------


Validation: 100%|██████████| 288/288 [00:39<00:00,  7.27it/s]


Train Loss: 0.109155 | Val Loss: 0.107641

Epoch 10/10
------------------------------------------------------------


Validation: 100%|██████████| 288/288 [00:39<00:00,  7.31it/s]

Train Loss: 0.109361 | Val Loss: 0.108409

TRAINING COMPLETE! Best Val Loss: 0.107568


In [ ]:
#model = torch.load('/kaggle/working/best_nfl_model.pth',weights_only="True")